In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
company_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.nodes.company.csv')

compete_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.compete.csv')
dispute_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.dispute.csv')

cooperate_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.cooperate.csv')
invest_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.invest.csv')
same_industry_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.same_industry.csv')
supply_df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/KnowledgeGraph/hidy.relationships.supply.csv')

In [ ]:
df_relation = pd.concat([compete_df, cooperate_df,dispute_df,invest_df,same_industry_df,supply_df])

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/submission/Task1.xlsx')

In [ ]:
df_example=df.head()

In [ ]:
df_example

,NewsID,NewsContent,Explicit_Company,label
0,66,全景网12月1日讯 以下为今日中国银行外汇牌价 货币名称现汇买入价现钞买入...,中国银行,0
1,421,全景网12月4日讯 以下为今日中国银行外汇牌价 货币名...,中国银行,0
2,435,中联重科控股子公司长沙浦沅进出口有限公司日前与印度ABG Heavy Industrie...,中联重科,0
3,708,全景网12月5日讯 以下为今日中国银行外汇牌价 货币名称现汇买入价现钞...,中国银行,0
4,787,“玉面书生”张良宾控制西昌电力的时代，终于在法律上宣告彻底结束。 今日，西昌电力公...,西昌电力,0


In [32]:
import time
from tqdm import tqdm

# 创建空列表来存储结果
df_t3_data = []

# 开始计时
start_time = time.time()

total_rows = len(df)
# 遍历 df 中的每一行
for index, row in tqdm(df.iterrows(), total=total_rows, desc="Processing"):
    explicit_company = row['Explicit_Company']
    explicit_company_id = None
    #found_explicit_company = False
    # 在 company_df 中查找 explicit_company 对应的 ID
    matches = company_df[company_df['company_name'] == explicit_company].reset_index(drop=True)

    if not matches.empty:

        explicit_company_id = matches.iloc[0][':ID']

        # 在 df_relation 中查找与 explicit_company_id 相关的信息
        matches = df_relation[(df_relation[':START_ID'] == explicit_company_id) | (df_relation[':END_ID'] == explicit_company_id)]
        implicit_pos = []
        implicit_neg = []
        if not matches.empty:
          for _, match_row in matches.iterrows():
              relationship_type = match_row[':TYPE']
              if match_row[':START_ID'] == explicit_company_id:
                  implicit_company_id = match_row[':END_ID']
              else:
                  implicit_company_id = match_row[':START_ID']

              # 使用 .item() 获取隐含公司的字符串值
              implicit_company = company_df[company_df[':ID'] == implicit_company_id]['company_name'].item()

              # 根据关系类型和标签进行处理
              if relationship_type in ['same_industry', 'cooperate', 'invest', 'supply']:
                  if row['label'] == 1:
                      implicit_pos.append(implicit_company)
                  else:
                      implicit_neg.append(implicit_company)
              elif relationship_type in ['compete', 'dispute']:
                  if row['label'] == 1:
                      implicit_neg.append(implicit_company)
                  else:
                      implicit_pos.append(implicit_company)

          new_row = {
              'NewsID': row['NewsID'],
              'NewsContent': row['NewsContent'],
              'Explicit_Company': explicit_company,
              'label': row['label'],
              'Implicit_Positive_Company': ', '.join(str(value) for value in implicit_pos),
              'Implicit_Negative_Company': ', '.join(str(value) for value in implicit_neg)
          }

          df_t3_data.append(new_row)

# 创建 DataFrame
df_t3 = pd.DataFrame(df_t3_data)

# 结束计时
end_time = time.time()
total_time = end_time - start_time
print(f"总计算时间: {total_time}秒")

Processing: 100%|██████████| 467426/467426 [1:17:50<00:00, 100.09it/s]


总计算时间: 4670.646958827972秒


In [31]:
df_t3.head

,NewsID,NewsContent,Explicit_Company,label,Implicit_Positive_Company,Implicit_Negative_Company
0,66,全景网12月1日讯 以下为今日中国银行外汇牌价 货币名称现汇买入价现钞买入...,中国银行,0,"山东黄金, 中国平安, 中国人保, 海通证券, 韵达股份, 浦发银行, 海通证券, 中信国安...","中国联通, 中国电信, 山东黄金, 中国铁建, 中国人寿, 中国能建, 农业银行, 华友钴业..."
1,421,全景网12月4日讯 以下为今日中国银行外汇牌价 货币名...,中国银行,0,"山东黄金, 中国平安, 中国人保, 海通证券, 韵达股份, 浦发银行, 海通证券, 中信国安...","中国联通, 中国电信, 山东黄金, 中国铁建, 中国人寿, 中国能建, 农业银行, 华友钴业..."
2,435,中联重科控股子公司长沙浦沅进出口有限公司日前与印度ABG Heavy Industrie...,中联重科,0,,"盈峰环境, 中国外运, 路畅科技, 汇川技术, 西部建设, 上海临港, 路畅科技, 路畅科技..."
3,708,全景网12月5日讯 以下为今日中国银行外汇牌价 货币名称现汇买入价现钞...,中国银行,0,"山东黄金, 中国平安, 中国人保, 海通证券, 韵达股份, 浦发银行, 海通证券, 中信国安...","中国联通, 中国电信, 山东黄金, 中国铁建, 中国人寿, 中国能建, 农业银行, 华友钴业..."


In [33]:
df_t3.to_excel('/content/drive/MyDrive/Colab_Notebooks/5002/dsaa5002_project/submission/Task2.xlsx', index=False)